In [1]:
!pip3 install xgboost scikit-learn numpy pandas torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

data_dir = 'data'
master_data_path = os.path.join(data_dir, 'ufc-master.csv')
df = pd.read_csv(master_data_path)
df.drop(columns=['date'], inplace=True)
df.drop(columns=['finish_round_time'], inplace=True)

categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
doesnt_need_ohe = df.drop(columns=categorical_columns)


ohe = OneHotEncoder(sparse_output=False)
needs_ohe = df[categorical_columns]

encoded_categories = pd.DataFrame(ohe.fit_transform(needs_ohe), columns=ohe.get_feature_names_out(categorical_columns))
encoded_df = pd.concat([doesnt_need_ohe, encoded_categories], axis=1)

In [3]:
# print("BEFORE PREPROCESSING")
# print(df.head())
# print("AFTER PREPROCESSING")
# print(encoded_df.head())


In [4]:
X = encoded_df.copy()
y = encoded_df['Winner_Blue']
X.drop(columns=['Winner_Blue', 'Winner_Red'], inplace=True)

rows, cols = X.shape


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31415, shuffle=True)

In [6]:
rf_classifier = RandomForestClassifier(n_estimators=cols, random_state=31415)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy {accuracy:.2f}')
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print('ROCAUC:', roc_auc_score(y_test, y_pred))

Accuracy 0.67
Classification Report:
              precision    recall  f1-score   support

         0.0       0.71      0.78      0.74       600
         1.0       0.58      0.50      0.54       380

    accuracy                           0.67       980
   macro avg       0.65      0.64      0.64       980
weighted avg       0.66      0.67      0.66       980

Confusion Matrix:
[[465 135]
 [190 190]]
ROCAUC: 0.6375000000000001


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the XGBoost classifier
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Train the model
xgb_clf.fit(X_train, y_train)
# Make predictions on the test set
y_pred = xgb_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

/home/linuxbrew/.linuxbrew/opt/python@3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/linuxbrew/.linuxbrew/opt/python@3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/linuxbrew/.linuxbrew/opt/python@3.11/lib/python3.11/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Accuracy: 63.47%


In [9]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy {accuracy:.2f}')
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print('ROCAUC:', roc_auc_score(y_test, y_pred))

Accuracy 0.63
Classification Report:
              precision    recall  f1-score   support

         0.0       0.70      0.71      0.70       600
         1.0       0.53      0.52      0.52       380

    accuracy                           0.63       980
   macro avg       0.61      0.61      0.61       980
weighted avg       0.63      0.63      0.63       980

Confusion Matrix:
[[426 174]
 [184 196]]
ROCAUC: 0.6128947368421053
